# December 22, 2022
https://adventofcode.com/2022/day/22


In [1]:
# too low: 7634

# Part 2:
# too low: 146241

In [106]:
import re
puz_fn = "../data/2022/22.txt"
test_fn = "../data/2022/22_test.txt"

In [83]:
def read_input( fn ):
    with open(fn, "r") as file:
        input = file.readlines()

    map = [ x.strip("\n") for x in input[:-2] ]
    
    # make map lines uniform length...
    mx = max( [len(line) for line in map] )
    map = [ line + "".join([" "]*(mx-len(line))) for line in map ]
    
    # parse instructions
    dir_string = input[-1]
    dir = []
    while True:
        ma = re.fullmatch( r"(\d+)([RL])(.*)", dir_string )
        if ma is None:
            break
        
        dir.append( [int(ma.group(1)), ma.group(2)] )
        dir_string = ma.group(3)

    dir.append( [int(dir_string), "X"] )
    return map, dir



In [129]:
test_map, test_dir = read_input(test_fn)
print("\n".join(test_map))
print(test_dir)

        ...#    
        .#..    
        #...    
        ....    
...#.......#    
........#...    
..#....#....    
..........#.    
        ...#....
        .....#..
        .#......
        ......#.
[[10, 'R'], [5, 'L'], [5, 'R'], [10, 'L'], [4, 'R'], [5, 'L'], [5, 'X']]


In [130]:
puz_map, puz_dir = read_input( puz_fn )

### Part 1

In [133]:
def follow_directions( map, dir, verbose=False ):
    ypos = 0
    xpos = 0
    while map[ypos][xpos] != ".":
        xpos += 1

    if verbose:
        print(f"Starting pos: ({xpos}, {ypos})")
    # List of [xstep, ystep] depending on bearing
    # Turning right goes to the next pair
    # Turning left goes to the prev pair
    steps = [ [1,0], [0,1], [-1,0], [0,-1] ]
    path = [">", "v", "<", "^"]

    bearing = 0
    xlast, ylast = xpos, ypos
    for dist, turn in dir:
        xstep, ystep = steps[bearing][0], steps[bearing][1]
        if verbose:
            print(f"Going {dist} steps with bearing {bearing} ({xstep}, {ystep})")
        i = 0
        while i < dist:
            xnew = (xpos + xstep) % len(map[0])
            ynew = (ypos + ystep) % len(map)

            # check for wall, in which case stop early
            if map[ynew][xnew] == "#":
                #print(f"Found wall at ({ynew}, {xnew})")
                # keep track of last floor tile, in case the wall is after blank spaces (i.e. wraparound)
                xpos = xlast
                ypos = ylast 
                break
            

            xpos = xnew
            ypos = ynew
            # only count the step if it wasn't a blank tile
            if map[ypos][xpos] != " ":
                map[ylast] = map[ylast][:xlast] + path[bearing] + map[ylast][(xlast+1):]
                xlast = xpos
                ylast = ypos
                i += 1


        # done moving, now turn:
        if verbose:
            print(f"New pos: ({xpos}, {ypos})")
            print(f"Turning {turn}")
        if turn == "R":
            bearing = (bearing + 1) % 4
        elif turn == "L":
            bearing = (bearing - 1) % 4

    map[ypos] = map[ypos][:xpos] + "@" + map[ypos][(xpos+1):]
        
       
    return xpos, ypos, bearing

In [134]:
test_map, test_dir = read_input(test_fn)
follow_directions(test_map, test_dir, verbose = True)

Starting pos: (8, 0)
Going 10 steps with bearing 0 (1, 0)
New pos: (10, 0)
Turning R
Going 5 steps with bearing 1 (0, 1)
New pos: (10, 5)
Turning L
Going 5 steps with bearing 0 (1, 0)
New pos: (3, 5)
Turning R
Going 10 steps with bearing 1 (0, 1)
New pos: (3, 7)
Turning L
Going 4 steps with bearing 0 (1, 0)
New pos: (7, 7)
Turning R
Going 5 steps with bearing 1 (0, 1)
New pos: (7, 5)
Turning L
Going 5 steps with bearing 0 (1, 0)
New pos: (7, 5)
Turning X


(7, 5, 0)

In [135]:
test_map

['        >>v#    ',
 '        .#v.    ',
 '        #.v.    ',
 '        ..v.    ',
 '...#...v..v#    ',
 '>>>v...@#.>>    ',
 '..#v...#....    ',
 '...>>>>v..#.    ',
 '        ...#....',
 '        .....#..',
 '        .#......',
 '        ......#.']

In [ ]:
puz_map, puz_dir = read_input( puz_fn )
follow_directions(puz_map, puz_dir, verbose=True)

In [138]:
(158+1)*1000 + (7+1)*4 + 2

159034

In [ ]:
print("\n".join(puz_map))

### Part 2

My map has these faces  
XAB  
XCX  
DEX  
FXX

A top -> F left  
A left -> D left  
B bottom -> C right  
B top -> F bottom  
B right E right  
C left  -> D top  
E bottom -> F right


In [149]:
def step_forward( xpos, ypos, xstep, ystep ):
    # going up the page:
    if xstep == 0 and ystep == -1:
        if ypos == 0 and (50 <= xpos < 100):
            # go off top A to left F:
            xnew = 0
            ynew = 150 + (xpos - 50)
            xstepnew = 1
            ystepnew = 0
        elif ypos == 0 and (100 <= xpos < 150):
            # go off top B to bot F
            xnew = (xpos-100)
            ynew = 199
            xstepnew = 0
            ystepnew = -1
        elif ypos == 100 and (0 <= xpos < 50):
            # go off top D to left C
            xnew = 50
            ynew = 50 + xpos
            xstepnew = 1
            ystepnew = 0
        else:
            # normal move
            xnew = xpos
            ynew = ypos - 1 
            xstepnew = xstep
            ystepnew = ystep

    # going right on page:
    elif xstep == 1 and ystep == 0:
        if xpos == 149 and (0 <= ypos < 50):
            # go off right B to right E
            xnew = 99
            ynew = 100 + (49-ypos)
            xstepnew = -1
            ystepnew = 0
        elif xpos == 99 and (50 <= ypos < 100):
            # go off right C to bot B
            xnew = 100 + (ypos - 50)
            ynew = 49
            xstepnew = 0
            ystepnew = -1
        elif xpos == 99 and (100 <= ypos < 150):
            # go off right E to right B
            xnew = 149
            ynew = (149 - ypos)
            xstepnew = -1
            ystepnew = 0
        elif xpos == 49 and (150 <= ypos < 200):
            # go off right F to bot E
            xnew = (ypos-150) + 50
            ynew = 149
            xstepnew = 0
            ystepnew = -1
        else:
            xnew = xpos + 1
            ynew = ypos
            xstepnew = xstep
            ystepnew = ystep

    # going down the page:
    elif xstep == 0 and ystep == 1:
        if ypos == 49 and (100 <= xpos < 150):
            # go off bot B to right C
            xnew = 99
            ynew = 50 + (xpos - 100)
            xstepnew = -1
            ystepnew = 0
        elif ypos == 149 and (50 <= xpos < 100):
            # go off bot E to right F
            xnew = 49
            ynew = 150 + (xpos - 50)
            xstepnew = -1
            ystepnew = 0
        elif ypos == 199 and (0 <= xpos < 50):
            # go off bot F to top B
            xnew = 100 + xpos
            ynew = 0
            xstepnew = 0
            ystepnew = 1
        else:
            xnew = xpos
            ynew = ypos + 1
            xstepnew = xstep
            ystepnew = ystep

    # going left on page
    elif xstep == -1 and ystep == 0:
        if xpos == 50 and (0 <= ypos < 50):
            # go off left A to left D
            xnew = 0
            ynew = 100 + (49 - ypos)
            xstepnew = 1
            ystepnew = 0
        elif xpos == 50 and (50 <= ypos < 100):
            # go off left C to top D
            xnew = (ypos - 50)
            ynew = 100
            xstepnew = 0
            ystepnew = 1
        elif xpos == 0 and (100 <= ypos < 150):
            # go off left D to left A
            xnew = 50
            ynew = (149 - ypos)
            xstepnew = 1
            ystepnew = 0
        elif xpos == 0 and (150 <= ypos < 200):
            # go off left F to top A
            xnew = (ypos - 150) + 50
            ynew = 0
            xstepnew = 0
            ystepnew = 1
        else:
            xnew = xpos - 1
            ynew = ypos
            xstepnew = xstep
            ystepnew = ystep

    return xnew, ynew, xstepnew, ystepnew

In [155]:
def follow_directions_now_in_3D( map, dir, verbose=False ):
    xpos = 50
    ypos = 0
    xstep = 1
    ystep = 0

    for dist, turn in dir:

        i = 0
        while i < dist:

            # draw current pos/bearing
            if xstep == 1:
                mark = ">"
            elif xstep == -1:
                mark = "<"
            elif ystep == 1:
                mark = "v"
            else:
                mark = "^"
            map[ypos] = map[ypos][:xpos] + mark + map[ypos][(xpos+1):]

            # take a step in the given direction
            if verbose:
                print(xpos, ypos, xstep, ystep, " ->  ", end="")
            xnew, ynew, xstepnew, ystepnew = step_forward( xpos, ypos, xstep, ystep )
            if verbose:
                print(xnew, ynew, xstepnew, ystepnew, end="")

            # check for wall, in which case stop early
            if map[ynew][xnew] == "#":
                # no move happened, don't update pos or bearing
                if verbose:
                    print(" #")
                break
            
            # no longer possible to step on blank spaces
            xpos = xnew
            ypos = ynew
            xstep = xstepnew
            ystep = ystepnew
            i += 1

        # steps = [ [1,0], [0,1], [-1,0], [0,-1] ]
        if turn == "R":
            if xstep != 0:
                ystep = xstep
                xstep = 0
            else:
                xstep = ystep * -1
                ystep = 0
        if turn == "L":
            if xstep != 0:
                ystep = xstep * -1
                xstep = 0
            else:
                xstep = ystep
                ystep = 0
    
    map[ypos] = map[ypos][:xpos] + "@" + map[ypos][(xpos+1):]
        
    if xstep == 1:
        bearing = 0
    elif ystep == 1:
        bearing = 1
    elif xstep == -1:
        bearing = 2
    elif ystep == -1:
        bearing = 3
    return xpos, ypos, bearing

In [156]:
puz_map, puz_dir = read_input( puz_fn )
follow_directions_now_in_3D(puz_map, puz_dir)

(60, 146, 1)

In [157]:
(146+1)*1000+(60+1)*4+1

147245

In [ ]:
puz_map